## Pics detection

### Rappel sur les strides

In [ ]:
import numpy as np

v=np.zeros([10],np.uint8)
print(v.strides)


v=np.zeros([10,15],np.uint8)
print(v.strides)

In [ ]:
v=np.zeros([10],np.float32)
print(v.strides)


v=np.zeros([10,15],np.float32)
print(v.strides)

### Make a rolling window

In [ ]:
def rolling_window(data, window_length):
    output_shape = data.shape[:-1] + (data.shape[-1] - window_length + 1, window_length)
    strides = data.strides + (data.strides[-1],)
    return np.lib.stride_tricks.as_strided(data, shape=output_shape, strides=strides)

In [ ]:
data=np.arange(0,11,1)
data

In [ ]:
roll=rolling_window(data,3)
roll

In [ ]:
roll.shape

***To you:*** $(2\heartsuit)$ Give the formula which links the shapes of  input and output of `rolling_window()`.  A drawing with a long vector, and a rolling window inside could help you to understand.

***To you:*** $(1\heartsuit)$ what does the `rolling_window()` function when the input `data` is not a vector, but a general `ndarray` (ex: a matrix).

### Version tensorflow

Voici une version faite à la main en tensorflow, essentiellement on utilise un reshape. Mais attention, en tensorflow, cela implique une copie des datas.

In [ ]:
import tensorflow as tf

def make_consecutive_windows(Z,window_size):
    def make_windows(Z, window_size):
        batch_size, nb_t = Z.shape[0], Z.shape[1]
        nb_windows = nb_t // window_size
        Z = Z[:, :window_size * nb_windows]
        Z = tf.reshape(Z, (batch_size, nb_windows, window_size) + Z.shape[2:])
        return Z

    res=[]
    nb_t=Z.shape[1]
    end_shape=Z.shape[2:]

    Z=Z[:,:nb_t // window_size * window_size - 1] #on racoursi un peu pour que tous les découpages aient le même nombre de tranche
    for i in range(window_size): #attention, on s'arrête avant window_size
        res.append(make_windows(Z[:,i:],window_size))
    res=tf.stack(res,axis=3) #

    bs,nb_windows=res.shape[0],res.shape[1]

    res=tf.reshape(res, (bs, nb_windows * window_size,window_size) + end_shape)
    return res

def test_consecutive_windows():
    """
    On teste avec des entrées de dimension 4 puis 3
    Le résultat doit être
    [[ 0.  1.  2.  3.  4.]
     [ 1.  2.  3.  4.  5.]
     [ 2.  3.  4.  5.  6.]
     [ 3.  4.  5.  6.  7.]
     [ 4.  5.  6.  7.  8.]
     [ 5.  6.  7.  8.  9.]
     [ 6.  7.  8.  9. 10.]
     [ 7.  8.  9. 10. 11.]
     [ 8.  9. 10. 11. 12.]
     [ 9. 10. 11. 12. 13.]
     [10. 11. 12. 13. 14.]
     [11. 12. 13. 14. 15.]
     [12. 13. 14. 15. 16.]
     [13. 14. 15. 16. 17.]
     [14. 15. 16. 17. 18.]]
    """
    b, nb_t, nb_part, dimX=1,20,1,1
    window_size=5

    T=tf.cast(tf.range(nb_t),tf.float32)
    X=tf.ones([b,nb_t , nb_part, dimX])

    X=X*T[None,:,None,None]
    print("X",X[0,:,0,0])

    X_w=make_consecutive_windows(X,window_size) #b,nb_window*window_size (all shift),window_size,nb_part,dimX
    print("X_w",X_w[0,:,:,0,0])

    dimY=1
    Y=tf.ones([b,nb_t,dimY])
    Y=Y*T[None,:,None]
    Y_w=make_consecutive_windows(Y,window_size)

    print("Y_w",Y_w[0,:,:,0])

test_consecutive_windows()

***A vous:*** Traduisez cette fonction en numpy.

### To find the local max

In [ ]:
def findLocalMax(data,windowLength):
    dataRoll=rolling_window(data,windowLength)
    print(dataRoll.shape)
    armax=np.argmax(dataRoll,axis=1)
    print(armax.shape)
    where=np.where(armax==(windowLength//2))
    print(where)
    return where[0]+windowLength//2

In [ ]:
data=np.ones(20)
data[0]=1.5
data[3]=2
data[4]=2.2
data[12]=1.1
plt.plot(range(len(data)),data,".");
plt.xticks(range(len(data)));

locMax=findLocalMax(data,5)
print(locMax)
plt.plot(locMax,data[locMax],"+");

***To you:***
* $(2\heartsuit)$ Explain how `findLocalMax()` works, in particular: What is the influence of the `windowLength` and why is it better to choose an odd integer.
* $(3\heartsuit)$ This function does not detect the picks if they are situated at the extremities of the `data` vector. Modify its, adding the optionnal arg `alsoDetectBorderPick=True`.

### On a physical signal

Here is a physical signal with two channels. With a simple observation left and right channels seem completly different.

In [ ]:
signal_phy, samplerate = sf.read('assets_signal/Sample.wav')
signal_phy.shape,samplerate

In [ ]:
def plot_a_part(x,y,index):
    fig,(ax0,ax1)=plt.subplots(2,1,figsize=(8,4),sharex=True)
    ax0.plot(x[:index],y[:index,0])
    ax1.plot(x[:index],y[:index,1])
    ax0.set_title("left")
    ax1.set_title("right")
    fig.tight_layout()

In [ ]:
N=len(signal_phy)
t=np.linspace(0,N*samplerate,N)

In [ ]:
plot_a_part(t,signal_phy,2000)

In [ ]:
plot_a_part(t,signal_phy,100)

The left channel is quite regular, while the right one move a lot. But let's observe their respective Fourier transform.

In [ ]:
spectrum_phy0=np.fft.rfft(signal_phy[:,0])
spectrum_phy1=np.fft.rfft(signal_phy[:,1])
spectrum_phy=np.stack([spectrum_phy0,spectrum_phy1],axis=1)
spectrum_phy.shape

Warning: the code

        spectrum_phy=np.fft.rfft(signal_phy)

would not gives what we expect (Grrrr).

In [ ]:
freqs=np.linspace(0,samplerate/2,len(spectrum_phy))
plot_a_part(freqs,np.abs(spectrum_phy),N//10)

In [ ]:
plot_a_part(freqs,np.abs(spectrum_phy),N//100)


We see that the two signals are actualy strongly correlated: they have the same pick of frequencies.

Explanation:

* the left signal is an electric signal, generated with an electronic device. Its is use to exitate a liquid

* the right signal is the output of a sensor dived in the liquid.


***To you:*** Detect the pics of both signals.
* $(2\heartsuit)$ Superpose this pic with the signals
* $(2\heartsuit)$  Superpose left and right pics to compare them


***To you:*** $(2\heartsuit)$ Describe a technic of extreme compression for sounds, which allows to say if two sounds are close or not.

Example: your technic could says if two signals are actually two recordings of the song "toxic" by "Britney Spear".

If you would had this idea before 2012, you would been the inventor of the famous software "shazam".   


